In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import psycopg2
import json

In [2]:
def get_today_data(search_symbol):
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    start_date = yesterday.strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    stock_data = yf.download(search_symbol, start = start_date, end = end_date).reset_index()
    return stock_data

In [3]:
def get_old_data(search_symbol):
    today = datetime.now()
    old = today - timedelta(days=730)
    start_date = old.strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    stock_data = yf.download(search_symbol, start = start_date, end = end_date).reset_index()
    return stock_data

In [4]:
with open('sql_cred.json') as f:
    credentials = json.load(f)

connection = psycopg2.connect(
    dbname=credentials['dbname'],
    user=credentials['user'],
    password=credentials['password'],
    host=credentials['host'],
    port=credentials['port']
)
cursor = connection.cursor()
#connection.rollback()

In [5]:
stock_list = pd.read_csv('Stocks_list.csv')
stock_list['search_symbol'] = stock_list.apply(lambda row: row['stock_symbol'] + ('.NS' if row['exchange'] == 'NSE' else '.BO'), axis=1)

In [6]:
stock_list.columns

Index(['stock_name', 'stock_symbol', 'exchange', 'sector', 'search_symbol'], dtype='object')

In [7]:
create_stock_list_table = f"""
CREATE TABLE IF NOT EXISTS "stock_list"(
    stock_name VARCHAR(70),
    stock_symbol VARCHAR(15),
    exchange VARCHAR(5),
    sector VARCHAR(30),
    UNIQUE (stock_symbol)
    );
"""
cursor.execute(create_stock_list_table)
for index, row in stock_list.iterrows():
    insert_query = f"""
        INSERT INTO "stock_list" (stock_name,stock_symbol, exchange, sector)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (stock_symbol) DO NOTHING;  -- Avoid inserting duplicates
        """
    cursor.execute(insert_query, (row['stock_name'], row['stock_symbol'], row['exchange'], row['sector']))

for index, row in stock_list.iterrows():
    stock_symbol = row.iloc[1]
    search_symbol = row.iloc[4]
    create_relevant_table = f'drop table if exists "{str(stock_symbol)}_fin"; '
    cursor.execute(create_relevant_table)

In [8]:
for index, row in stock_list.iterrows():
    stock_symbol = row.iloc[1]
    search_symbol = row.iloc[4]
    create_relevant_table = f"""
    CREATE TABLE IF NOT EXISTS "{str(stock_symbol)}_fin" (
        date DATE,
        symbol VARCHAR(20),
        open DECIMAL(10, 2),
        high DECIMAL(10, 2),
        low DECIMAL(10, 2),
        close DECIMAL(10, 2),
        adj_close DECIMAL(10, 2),
        volume BIGINT,
        UNIQUE (date)
    );
    
    ALTER TABLE "{str(stock_symbol)}_fin" OWNER TO sunny;
    """
    
    cursor.execute(create_relevant_table)
    data = get_old_data(search_symbol)
    for index, row in data.iterrows():
        insert_query = f"""
        INSERT INTO "{str(stock_symbol)}_fin" (date,symbol, open, high, low, close, adj_close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (date) DO NOTHING;  -- Avoid inserting duplicates
        """
        cursor.execute(insert_query, (row['Date'],stock_symbol, row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume']))

[*********************100%%**********************]  1 of 1 completed

/home/sunny/anaconda3/envs/autoTrade/lib/python3.12/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed

/home/sunny/anaconda3/envs/autoTrade/lib/python3.12/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed

...

/home/sunny/anaconda3/envs/autoTrade/lib/python3.12/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed

/home/sunny/anaconda3/envs/autoTrade/lib/python3.12/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed


/home/sunny/anaconda3/envs/autoTrade/lib/python3.12/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [9]:
connection.commit()
cursor.close()
connection.close()

In [17]:
import requests
import pandas as pd
from io import StringIO

In [18]:
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vR1hX501C2zCJxNBrNw3ZhRmKgi3YmEGtajMSTUmPcXuDkVDVh5TZ9ZAVr8MsYJ3YC-SOpIicOfmtn5/pub?gid=0&single=true&output=csv"
# Download the CSV file content
response = requests.get(sheet_url)

In [19]:
# Check if the request was successful
if response.status_code == 200:
    # Convert the CSV content to a Pandas DataFrame
    data = StringIO(response.text)
    stock_data = pd.read_csv(data)
    
    # Display the first few rows of the data
    print(stock_data.head())
else:
    print(f"Failed to download the file, status code: {response.status_code}")

                      stock_name stock_symbol exchange                  sector
0  Tata Consultancy Services Ltd          TCS      NSE  Software & IT Services
1                    Infosys Ltd         INFY      NSE  Software & IT Services
2           HCL Technologies Ltd      HCLTECH      NSE  Software & IT Services
3                  HDFC Bank Ltd     HDFCBANK      NSE                   Banks
4                 ICICI Bank Ltd    ICICIBANK      NSE                   Banks


In [21]:
stock_data.to_csv('Stock_list.csv', index=False)